In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
Full={}
Full_f={}
Full_f1={}
max_kin=2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os,sys
import glob
import numpy as np
import tabulate
from tabulate import tabulate
from numpy import *
import scipy
from scipy import interpolate
from scipy.optimize import curve_fit

from scipy.interpolate import make_interp_spline, BSpline
from scipy.interpolate import griddata

sys.path.append('/home/jbane/headers')
import SQLFuncs as SQLF
import seaborn as sns


plt.style.use('seaborn-white')

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['figure.titlesize'] = 16



def Cross_table(tgt,table):
   
    #Fill the table once
    if len(table) != 10:
        for i in range(16):
            file_name = "/home/jbane/tritium/T2_externals/OUT/marathon_kin{}_{}.out".format(i,tgt)
            #file_name = "/home/jbane/tritium/extern/T2_externals/OUT/marathon_kin{}_{}.out".format(i,tgt)
           # print(file_name)
            if os.path.isfile(file_name):
                table[i] = pd.read_table(file_name,delim_whitespace=True)

def whatangle(kin):
    KT=kintable()
    if type(kin) is not str:
        j=str(kin)
    theta = KT.query("kin == '{}'".format(j))['theta'].values[0]
    return theta
    
def kintable():
    
    file ="/home/jbane/tritium/T2_externals/marathon_kin_table.csv"
    kintable= pd.read_csv(file)
    A=kintable.keys()
    mp=0.93828
    kintable["Q2"]= 4 *kintable[A[2]]*kintable["E "]*sin(kintable["theta"]/2 * np.pi/180)**2
    kintable["X"] = kintable["Q2"]/(2*mp * (kintable["E "]-kintable[A[2]]) )
    kintable["W2"] =mp**2 - kintable["Q2"] + (2*mp * (kintable["E "]-kintable[A[2]])) 
    kintable=kintable.query('index<10')  
    return kintable
                
def interper(DFs,x,Q):
    f = interpolate.interp2d(DFs["x"],DFs["Q2"],DFs["Sig_Born"],kind='linear', copy=True,  bounds_error=True)
    sigma_born=f(x,Q)
    h = interpolate.interp2d(DFs["x"],DFs["Q2"],DFs["Sig_Rad"],kind='linear', copy=True, bounds_error=True )
    sigma_rad=h(x,Q)
    return sigma_born,sigma_rad  

def weighted_average(df,data_col,weight_col,by_col):
    df['_data_times_weight'] = df[data_col]*df[weight_col]
    df['_weight_where_notnull'] = df[weight_col]*pd.notnull(df[data_col])
    g = df.groupby(by_col)
    result = g['_data_times_weight'].sum() / g['_weight_where_notnull'].sum()
    del df['_data_times_weight'], df['_weight_where_notnull']
    return result

def wavg(group, avg_name, weight_name):
    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()
    
def inquad(A,B):
    C=sqrt(pow(A,2)+pow(B,2))
    return C

def inquads(A):
    C=0
    for a in A:
        C=sqrt(pow(C,2)+pow(a,2))
    return C

   
def timefromfill(run):
    
    strtime= SQLF.starttime(run)
    filltime = datetime.datetime(2017,11,23)
    TD = strtime-filltime
    return TD.total_seconds()
    

markers =[[3,0,0] ,[5,1,90] ,[4,0,90],[3,1,90],[6,2,0] ,[5,0,0],[4,1,0],[7,0,0],[3,0,0] ,[5,1,90]]
colors=['red', 'Green', 'Blue','Maroon', 'Orange', 'Purple', 'Cyan', 'Magenta','Teal','Brown','Navy']



cuttype="tightcut"
xaxis="xbjc"
BlankDF = {1: pd.DataFrame()}
debug=0

def Yield(tgt="D2",bins="2",suf="all",xaxis="Xbjc",onekin="",He3DF=BlankDF,plot=0):
    dataDF={}
    mcDFs={}
    dataDFs={}
    ratios={}
    DF={}
        
    if tgt == "H3":
        dobeta=True
    else:
        dobeta=False
      
    

    xfolder ="xbj"
    subdir=""

    if xaxis != "Xbjc":
        xfolder="theta"
        
    if xaxis == "E`":
        subdir = "_e"
    
    mcfile='/home/jbane/tritium/Tri_offline/MC_comparison/yield_output/{}/{}bins/{}/'.format(cuttype,bins,xfolder)
    datafile='/home/jbane/tritium/Tri_offline/yield/yield_output/{}/{}bins/{}/'.format(cuttype,bins,xfolder)
    kins={}
    kin={}
    mckin={}
    datakin={}
    sig_r={}
    sig_b={}
    sig_rr={}
    sig_bb={}
    mcDF={}
 #   dataDF={}
 #   mcDFs={}
 #   dataDFs={}
  #  ratios={}
  #  DF={}
    
    kins=sorted(glob.glob('{}{}_kin?{}.dat'.format(mcfile,tgt,subdir)))
    kins+=sorted(glob.glob('{}{}_kin??{}.dat'.format(mcfile,tgt,subdir)))
    i=0
  
    for kinfile in kins:
        mckin[i]=kinfile.replace(mcfile,"")
        mckin[i]=mckin[i].replace(tgt,"")
        mckin[i]=mckin[i].replace("_kin","")
        if xaxis == "E`":
            mckin[i]=mckin[i].replace(subdir,"")
        mckin[i]=mckin[i].replace(".dat","")
        i=i+1    

    kins=sorted(glob.glob('{}{}_kin?{}.dat'.format(datafile,tgt,subdir)))
    kins+=sorted(glob.glob('{}{}_kin??{}.dat'.format(datafile,tgt,subdir)))

    i=0
    for kinfile in kins:
        datakin[i]=kinfile.replace(datafile,"")
        datakin[i]=datakin[i].replace(tgt,"")
        datakin[i]=datakin[i].replace("_kin","")
        if xaxis == "E`":
            datakin[i]=datakin[i].replace(subdir,"")
        datakin[i]=datakin[i].replace(".dat","")
        i=i+1

    if mckin == datakin:
        kin=mckin.copy()
    else :
        for a in range(0,len(mckin)):
            if mckin[a] in datakin:
                kin[a] = mckin[a]

    #fig1= plt.figure(dpi=500,figsize=(12,12))
    #ax=fig1.add_subplot(1,1,1)

    fig= plt.figure(dpi=300,figsize=(12,9))
    nog=2
    zx=fig.add_subplot(nog,1,1)
    plt.title('{} Data to MC comparison'.format(tgt))
    bx=fig.add_subplot(nog,1,2, sharex=zx)
    #cx=fig.add_subplot(nog,1,3, sharex=zx)
    #dx=fig.add_subplot(nog,1,4, sharex=zx)
    #ex=fig.add_subplot(nog,1,3, sharex=zx)
    markers =[[3,0,0]  ,[5,1,90] ,[4,0,90],[3,1,90],[6,2,0] ,[5,0,0] ,[4,1,0],[7,0,0],[7,2,90],[3,1,90],[7,2,45],[3,1,90],[8,0,0]]
    colors=['red', 'Green', 'Blue', 'Orange', 'Purple', 'Cyan', 'Magenta','Teal','Navy','Brown','Mint','Maroon']
    i=0

    if onekin != '':
        kin.clear()
        onekins=onekin.split(" ")
        jj=0
        for ok in onekins:
            
            kin[jj]=ok
            jj=jj+1
        print(jj, ok , kin)
        #kin={0:onekin}

    #fig2= plt.figure(dpi=100,figsize=(9,5))
    #new=fig2.add_subplot(1,1,1)

    for kinnum in kin.values():
            kinint=int(kinnum)
            mcfileR=mcfile+'{}_kin{}{}.dat'.format(tgt,kinnum,subdir)
            datafileR=datafile+'{}_kin{}{}.dat'.format(tgt,kinnum,subdir)       
            
            mcDF[kinint]=pd.read_table(mcfileR,delim_whitespace=True,dtype=float)

            dataDF[kinint]=pd.read_table(datafileR,delim_whitespace=True,dtype=float)
            mcDFs[kinint]=mcDF[kinint].dropna().copy()
            dataDFs[kinint]=dataDF[kinint].dropna().copy()

            if debug >=1:
                print(dataDF[kinint])
                print(mcDF[kinint])
            
            dataDFs[kinint]['Stat Err'] = 1/np.sqrt(dataDFs[kinint]["Ne"]) * dataDFs[kinint]['Yield']

            totallum,lumerr,thick_err,denscor,denscor_err=SQLF.total_lum(tgt,kinint,suf)

            charge_err = 0.002
            
            dataDFs[kinint]['Lumin']=totallum
            dataDFs[kinint]['Lumin Err']=lumerr
            dataDFs[kinint]['Thickness Err per']=round(thick_err*100,3)
            dataDFs[kinint]['Dens Cor. Err per']=round(denscor_err,3)
            dataDFs[kinint]['Dens Cor.']=round(denscor,3)
            dataDFs[kinint]['Charge Err per']=round(charge_err,3)
            
            dataDFs[kinint]['Lumin Err per'] = sqrt(pow(dataDFs[kinint]['Dens Cor. Err per'],2)
                                                   +pow(dataDFs[kinint]['Charge Err per'],2))
            
            
            
            if xaxis=="Xbjc":
                PC,PCerr=SQLF.PC(tgt,dataDFs[kinint]['Xbjc'])
                dataDFs[kinint]['PC']= 1 - PC
                dataDFs[kinint]['PC Err']=PCerr
            elif xaxis =="E`":
                dataDFs[kinint]['E`']= round(dataDFs[kinint]['E`'].copy()*100,4)
                mcDFs[kinint]['E`']= round(mcDFs[kinint]['E`'].copy(),4)
                PC,PCerr =SQLF.PC(tgt,dataDFs[kinint]['xbj']) 
                dataDFs[kinint]['PC']= 1-PC
                dataDFs[kinint]['PC Err']=PCerr
            elif xaxis =="theta":
                PC,PCerr =SQLF.PC(tgt,dataDFs[kinint]['xbj']) 
                dataDFs[kinint]['PC']= 1-PC
                dataDFs[kinint]['PC Err']=PCerr
            
            
            ECC = SQLF.ECC(tgt,kinint)
            dataDFs[kinint]['ECC']= 1 - ECC
            dataDFs[kinint]['ECC Err'] = 0.007

            RunC = SQLF.total_eff(tgt,kinint,suf)
            dataDFs[kinint]["Effs"] = RunC[0]
            dataDFs[kinint]["Eff err"] = RunC[1]

            dataDFs[kinint]["Yield_b_lum"] = dataDFs[kinint]["Ne"]/dataDFs[kinint]["Lumin"]
            dataDFs[kinint]["Yield_b_lum_eff"]=dataDFs[kinint]["Yield_b_lum"]/dataDFs[kinint]["Effs"]
            dataDFs[kinint]["Yield_b_lum_eff_BG"]=(dataDFs[kinint]["Yield_b_lum"]/dataDFs[kinint]["Effs"]) * 1/(dataDFs[kinint]['ECC']*dataDFs[kinint]["PC"])

            #dataDFs[kinint]['Yield_Bin']=dataDFs[kinint]["Ne"]
            mcDFs[kinint]=mcDFs[kinint].rename(index=int,columns={"Ne":"MC_Ne","Yield":"MC_Yield","Error":"MC_Error"})
            if xaxis=="Xbjc":
                mcDFs[kinint]=mcDFs[kinint].drop(columns={'Xbj'})
                
            mcDFs[kinint]['MC_Yield'] =mcDFs[kinint]['MC_Yield']/1000
            mcDFs[kinint]['MC_Error'] =mcDFs[kinint]['MC_Error']/1000

            frames=[mcDFs[kinint],dataDFs[kinint]];

            
           
                        
            if xaxis=="E`":
                1
                #for j in range(len(mcDFs[kinint])):
                #    if mcDFs[kinint]['E`'][j]!=dataDFs[kinint]['E`'][j]:
                #        dataDFs[kinint]['E`'][j] =mcDFs[kinint]['E`'][j].copy()

            
            #DF[kinint]=pd.concat(frames)#,axis=0,ignore_index=True)
            if xaxis =="Xbjc":
                DF[kinint]=pd.merge(dataDFs[kinint],mcDFs[kinint], on="Xbjc")
            else :
                DF[kinint]=pd.merge(dataDFs[kinint],mcDFs[kinint], on="{}".format(xaxis))
                
            DF[kinint].index=dataDFs[kinint].index

            #Drop last bin
            DF[kinint]=DF[kinint].drop(DF[kinint].index[len(DF[kinint].index)-1])
            
           # if xaxis =="Xbjc":
           #     if kinint == 5:
           #         DF[kinint]=DF[kinint].query("Xbjc<=0.4")
           #     if kinint == 7:
           #         DF[kinint]=DF[kinint].query("Xbjc>=0.4")
            DF[kinint]["StEp"]= DF[kinint]["Stat Err"]/DF[kinint]['Yield']    
            DF[kinint]=DF[kinint].query("StEp < 0.075")

                    
                    
            DF[kinint]['ratio b lum'] =DF[kinint]['Yield_b_lum']/DF[kinint]['MC_Yield']
            DF[kinint]['ratio b lum eff'] =DF[kinint]['Yield_b_lum_eff']/DF[kinint]['MC_Yield']
            DF[kinint]['ratio b lum eff BG'] =DF[kinint]['Yield_b_lum_eff_BG']/DF[kinint]['MC_Yield']

            DF[kinint]['Data total Yield Error'] =  DF[kinint]["Yield"]* sqrt(pow(DF[kinint]['PC Err']/DF[kinint]['PC'],2)  
                                                + pow(DF[kinint]["Eff err"]/DF[kinint]["Effs"],2)
                                                + pow(DF[kinint]['Stat Err']/DF[kinint]['Yield'],2)
                                                + pow(DF[kinint]['ECC Err'],2)
                                                + pow(DF[kinint]['Charge Err per'],2)
                                                + pow(DF[kinint]['Dens Cor. Err per'],2) )
                                                # pow(DF[kinint]['Lumin Err']/DF[kinint]['Lumin'],2)           
                

                
            if tgt == "H3" and dobeta:
                Betaf = He3ContKin(str(kinint),suf)
                #print(Betaf)
                DF[kinint]["Cont. Yield"] =DF[kinint]['Yield']
                DF[kinint]['Yield']   =  Betaf[0]*DF[kinint]['Yield'] - Betaf[1]*He3DF[kinint]['Yield']
              #  DF[kinint]['Yield'] = DF[kinint]['Yield']*(1 + Betaf[4])
                
                deltaA  = DF[kinint]['Cont. Yield'] * Betaf[0] *np.sqrt((Betaf[2]/Betaf[0])**2 + ( DF[kinint]['Data total Yield Error']/DF[kinint]['Yield'])**2) 
                deltaB  = He3DF[kinint]['Yield']*Betaf[1]*np.sqrt((Betaf[3]/Betaf[1])**2 + (He3DF[kinint]['Data total Yield Error']/He3DF[kinint]['Yield'])**2)
                T_delta_per = np.sqrt(deltaA**2+deltaB**2)
                #print(T_delta_per)
                DF[kinint]['Cont. Data total Yield Error'] =DF[kinint]['Data total Yield Error']
                DF[kinint]['Data total Yield Error'] = DF[kinint]['Yield']*np.sqrt(DF[kinint]['Data total Yield Error']**2 + (T_delta_per/DF[kinint]['Yield'])**2)
                DF[kinint]["Beta Decay Error per."] =(T_delta_per/DF[kinint]['Yield'])
            else :
                DF[kinint]["Beta Decay Error per."] =0.00
                DF[kinint]["Cont. Yield"] =DF[kinint]['Yield']
                DF[kinint]['Cont. Data total Yield Error'] =DF[kinint]['Data total Yield Error']
            
            
            DF[kinint]['D-mc ratio cont'] =DF[kinint]['Yield']/DF[kinint]['MC_Yield']
            DF[kinint]['D-mc ratio'] =DF[kinint]['Yield']/DF[kinint]['MC_Yield']
            DF[kinint]['Total D-mc error'] =  sqrt( pow(DF[kinint]['Data total Yield Error']/DF[kinint]["Yield"],2) + pow(DF[kinint]['MC_Error']/DF[kinint]['MC_Yield'],2) )
            
                       
            
            DF[kinint]=DF[kinint].dropna()
            #DF[kinint]=DF[kinint].drop(DF[kinint][DF[kinint]["Ne"]<=200].index)
            DF[kinint]['kin']=kinint
            DF[kinint]['D-mc stat error'] =DF[kinint]['D-mc ratio'] *inquad(DF[kinint]['MC_Error']/DF[kinint]['MC_Yield'],DF[kinint]['Stat Err']/DF[kinint]['Yield'])

            DF[kinint]['D-mc error noeff'] =DF[kinint]['D-mc ratio'] * sqrt(  
                                                  pow(DF[kinint]['PC Err']/DF[kinint]['PC'],2)  
                                                + pow(DF[kinint]['Stat Err']/DF[kinint]['Yield'],2)
                                                + pow(DF[kinint]['ECC Err'],2)
                                                + pow(DF[kinint]['Charge Err per'],2)
                                                + pow(DF[kinint]['Dens Cor. Err per'],2) )
                                                 #pow(DF[kinint]['Lumin Err']/DF[kinint]['Lumin'],2) 
            
            if bins =="50":
                DF[kinint]=DF[kinint].drop(DF[kinint].index[0])    
                DF[kinint]=DF[kinint].drop(DF[kinint].index[len(DF[kinint].index)-1])
                DF[kinint]=DF[kinint].drop(DF[kinint].index[len(DF[kinint].index)-1])
            elif int(bins) < 15:
            #    DF[kinint]=DF[kinint].drop(DF[kinint].index[0])    
            #    DF[kinint]=DF[kinint].drop(DF[kinint].index[len(DF[kinint].index)-1])
            #    print("Droping extreems")
            #    DF[kinint]=DF[kinint].drop(DF[kinint].index[( (DF[kinint]["D-mc stat error"]) > 0.05)])
                ccc=2
            else :
            #    DF[kinint]=DF[kinint].drop(DF[kinint].index[( (DF[kinint]["D-mc stat error"]) > 0.05)])
                ccc=2

            #DF[kinint]=DF[kinint].drop(DF[kinint].index[0])

            
           # print(DF[kinint]['Xbjc'])
        
            if(len(DF[kinint][xaxis]) <1):
                continue

            DF[kinint].dropna()
            
           
            if plot==1:
                if xaxis == "xbj":
                    if kinnum == kin[0]:
                        DF[kinint].plot(ax=zx,y='Yield',x='{}'.format(xaxis), yerr='Data total Yield Error',kind='scatter',c='r',marker=markers[i],label='Data' )
                        DF[kinint].plot(ax=zx,y='MC_Yield',x='{}'.format(xaxis), yerr='MC_Error',kind='scatter',c='g',marker=markers[i],label='MC' )
                     #   DF[kinint].plot(ax=zx,y='Yield_b_lum',x='Xbjc', yerr='Stat Err',kind='scatter',c='cyan',marker=markers[i],label='Data binwise' )
                     #   DF[kinint].plot(ax=zx,y='Yield_b_lum_eff',x='Xbjc', yerr='Stat Err',kind='scatter',c='navy',marker=markers[i],label='Data binwise & eff' )
                     #   DF[kinint].plot(ax=zx,y='Yield_b_lum_eff_BG',x='{}'.format(xaxis), yerr='Stat Err',kind='scatter',c='b',marker=markers[i],label='Data binwise & eff & BG' )

                    else :
                        DF[kinint].plot(ax=zx,y='Yield',x='{}'.format(xaxis), yerr='Data total Yield Error',kind='scatter',c='r',marker=markers[i] )
                        DF[kinint].plot(ax=zx,y='MC_Yield',x='{}'.format(xaxis), yerr='MC_Error',kind='scatter',c='g',marker=markers[i] )
                        #DF[kinint].plot(ax=zx,y='Yield_b_lum',x='Xbjc', yerr='Stat Err',kind='scatter',c='cyan',marker=markers[i] )
                        #DF[kinint].plot(ax=zx,y='Yield_b_lum_eff',x='Xbjc', yerr='Stat Err',kind='scatter',c='navy',marker=markers[i])
                       # DF[kinint].plot(ax=zx,y='Yield_b_lum_eff_BG',x='{}'.format(xaxis), yerr='Stat Err',kind='scatter',c='b',marker=markers[i])

                    DF[kinint].plot(ax=bx,y='D-mc ratio',x='{}'.format(xaxis), yerr='Total D-mc error',kind='scatter',c=colors[i],marker=markers[i],label="kin {}".format(kinint) )
                    #DF[kinint].plot(ax=cx,y='ratio b lum',x='Xbjc', yerr='D-mc error',kind='scatter',c=colors[i],marker=markers[i] )
                    #DF[kinint].plot(ax=dx,y='ratio b lum eff',x='Xbjc', yerr='D-mc error',kind='scatter',c=colors[i],marker=markers[i],label="kin {}".format(kinint) )
                    #DF[kinint].plot(ax=ex,y='ratio b lum eff BG',x='{}'.format(xaxis), yerr='D-mc error',kind='scatter',c=colors[i],marker=markers[i] )
                    #DF[kinint].plot(ax=bx,y='ratio b lum eff BG',x='Xbjc', yerr='D-mc error',kind='scatter',c=colors[i],marker=markers[i],label="kin {}".format(kinint) )

                else :

                    if kinnum == kin[0]:
                        DF[kinint].plot(ax=zx,y='Yield',x='{}'.format(xaxis), yerr='Error',kind='scatter',c='r',marker=markers[i],label='Data')
                        DF[kinint].plot(ax=zx,y='MC_Yield',x='{}'.format(xaxis), yerr='MC_Error',kind='scatter',c='g',marker=markers[i],label='MC' )
                     #   DF[kinint].plot(ax=zx,y='Yield_b_lum_eff_BG',x='{}'.format(xaxis), yerr='Stat Err',kind='scatter',c='b',marker=markers[i],label='Data binwise & eff & BG' )
                    else :
                        DF[kinint].plot(ax=zx,y='Yield',x='{}'.format(xaxis), yerr='Error',kind='scatter',c='r',marker=markers[i])
                        DF[kinint].plot(ax=zx,y='MC_Yield',x='{}'.format(xaxis), yerr='MC_Error',kind='scatter',c='g',marker=markers[i] )
                       # DF[kinint].plot(ax=zx,y='Yield_b_lum_eff_BG',x='{}'.format(xaxis), yerr='Stat Err',kind='scatter',c='b',marker=markers[i] )
                    DF[kinint].plot(ax=bx,y='D-mc ratio',x='{}'.format(xaxis), yerr='Total D-mc error',kind='scatter',c=colors[i],marker=markers[i],label="kin {}".format(kinint) )
                   # DF[kinint].plot(ax=bx,y='ratio b lum eff BG',x='{}'.format(xaxis), yerr='D-mc stat error',kind='scatter',c=colors[i],marker=markers[i] )         
                i=i+1    
                
            #DF[kinint]=dataDFs[kinint].append(mcDFs[kinint])
            #pd.concat([mcDFs[kinint],dataDFs[kinint]])
         #   print(DF[kinint]['Xbjc'])
    zx.set_yscale("log")   
    zx.set_ylabel("Yield")
    zx.grid(True)
    #ax.set_yscale("log")
    #ax.grid(True)
    bx.grid(True)
    bx.set_ylabel("Data / MC ratio event")
    if xaxis=="Xbjc":
        bx.set_xlabel("$X_{BJ}$")
    else :
        bx.set_xlabel(xaxis)
   # cx.grid(True)
    #dx.grid(True)
    #ex.grid(True)
    #ex.set_ylabel("Data / MC ratio Binwise")
    if xaxis=="Xbjc":
        zx.set_xlabel("$X_{BJ}$")
    elif xaxis =="E`" :
        bx.set_xlabel( "Delta %")
    bx.set_ylim(0.82,1.12)
    #cx.set_ylim(0.86,1.08)
    #dx.set_ylim(0.86,1.12)
    #ex.set_ylim(0.82,1.12)

 #  new.grid(True)
 #   new.legend(ncol=4)
 #   new.set_ylabel("Data/MC for {}".format(tgt))
 #   print("::",DF[1]['Xbjc'])
    zx.legend(ncol=4)
    bx.legend(ncol=4)
    #dx.legend(ncol=4)
    fig.subplots_adjust(hspace=0)   

    #print()
    #print()
   # print(DF[1].keys())
    #print()
    print()
    
    
    return DF,dataDFs,mcDFs,fig
     
def YieldFull(DF,xaxis="Xbjc"):
    Fulltmp = pd.concat(DF,keys=DF.keys())
    cname =Fulltmp.columns
    numofc=len(cname)
    ab={}

    for i,col in enumerate(cname):
        ab[i]=Fulltmp.groupby(level=1).apply(wavg,col ,"Ne")
    Full= pd.DataFrame.from_dict(ab)
    Full.columns = cname
    fig= plt.figure(dpi=300,figsize=(12,7))
    ax=fig.add_subplot(1,1,1)

    Full.plot(ax=ax,x=xaxis,y="D-mc ratio",yerr='D-mc stat error',kind='scatter',c="r", label="Beta Decay Corrected")
    #Full.plot(ax=ax,x=xaxis,y="ratio b lum eff BG",yerr='D-mc stat error',marker="d",kind='scatter',color="g",label="No Beta Decay")
    ax.set_ylabel("Data / MC ratio")
    if xaxis == "xbj" or xaxis == "Xbjc" :
        ax.set_xlabel("$X_{BJ}$")
        ax.set_ylim(0.85,1.15)   
    else :
        ax.set_xlabel(xaxis)
        
    ax.grid("True")
     
    return Full

def YieldEMC(FullA,atgt,a,FullB,btgt,b):
    
    As=FullA[['Xbjc','Yield','Stat Err','MC_Yield','MC_Error','D-mc ratio']].copy()
    anames=As.keys() + atgt
    As.columns=anames
    As=As.rename(index=str,columns={"Xbjc{}".format(atgt):"Xbjc"})

    Bs=FullB[['Xbjc','Yield','Stat Err','MC_Yield','MC_Error','D-mc ratio']].copy()
    bnames=Bs.keys() + btgt
    Bs.columns=bnames
    Bs=Bs.rename(index=str,columns={"Xbjc{}".format(btgt):"Xbjc"})

    EMC=pd.merge(As,Bs, on="Xbjc")
    EMC["EMC yield"] = EMC["Yield{}".format(atgt)]/EMC["Yield{}".format(btgt)] * b/a
    EMC["EMC yield err"] = np.sqrt(pow(EMC['Stat Err{}'.format(atgt)]/EMC['Yield{}'.format(atgt)],2) + pow(EMC['Stat Err{}'.format(btgt)]/EMC['Yield{}'.format(btgt)],2) )

    fig = plt.figure(dpi=300,figsize=(12,7))
    ax=fig.add_subplot(1,1,1)
    EMC.plot(ax=ax,y='EMC yield',x="Xbjc",yerr="EMC yield err",kind='scatter')

    ax.grid("True")
    #ax.set_ylim(0.75,1.2)
    return EMC["EMC yield"],EMC["EMC yield err"]
    
def BornEMC(FullA,atgt,a,FullB,btgt,b):
    
    As=FullA[['Xbjc','Yield','Stat Err','MC_Yield','MC_Error','D-mc ratio','D-mc error','Thickness Err per']].copy()
    anames=As.keys() + atgt
    As.columns=anames
    As=As.rename(index=str,columns={"Xbjc{}".format(atgt):"Xbjc"})

    Bs=FullB[['Xbjc','Yield','Stat Err','MC_Yield','MC_Error','D-mc ratio','Thickness Err per']].copy()
    bnames=Bs.keys() + btgt

    Bs.columns=bnames
    Bs=Bs.rename(index=str,columns={"Xbjc{}".format(btgt):"Xbjc"})

    EMC=pd.merge(As,Bs, on="Xbjc")
    EMC["EMC yield"] = EMC["Yield{}".format(atgt)]/EMC["Yield{}".format(btgt)] * b/a
    EMC["EMC yield err"] = np.sqrt(pow(EMC['Stat Err{}'.format(atgt)]/EMC['Yield{}'.format(atgt)],2) + pow(EMC['Stat Err{}'.format(btgt)]/EMC['Yield{}'.format(btgt)],2) )

    fig = plt.figure(dpi=300,figsize=(12,7))
    ax=fig.add_subplot(1,1,1)
    EMC.plot(ax=ax,y='EMC yield',x="Xbjc",yerr="EMC yield err",kind='scatter')

    ax.grid("True")
def torad(t) :
    return t*np.pi/180
def todeg(t):
    return t*180/np.pi   
    
def Born(DF, tgt):
    CST={}
    DFBorn1={}
    DFBorn2={}
    DFBorn3={}
    Cross_table(tgt,CST)
    
    KT=kintable()
    for j in DF.keys():
        
        if 'Xbjc' and 'Q2' in DF[j].keys():
            xbj=DF[j]["Xbjc"]
            Q2=DF[j]["Q2"]
            Born1= griddata( (CST[j]['x'],CST[j]['Q2']),CST[j]['Sig_Born'] ,(xbj,Q2) ,method='cubic')
            CC = griddata( (CST[j]['x'],CST[j]['Q2']),CST[j]['C_cor'] ,(xbj,Q2) ,method='cubic')
            RC = griddata( (CST[j]['x'],CST[j]['Q2']),CST[j]['Sig_Rad'] ,(xbj,Q2) ,method='cubic')
        elif 'E`' in DF[j].keys():
            E=DF[j]["E`"]
            theta = KT.query("kin == '{}'".format(j))['theta'].values[0]           
            Born1= griddata( (CST[j]['Eprime'],CST[j]['Theta']),CST[j]['Sig_Born'] ,(E,theta) ,method='cubic')
            CC= griddata( (CST[j]['Eprime'],CST[j]['Theta']),CST[j]['C_cor'] ,(E,theta) ,method='cubic')
            RC = griddata( (CST[j]['Eprime'],CST[j]['Theta']),CST[j]['Sig_Rad'] ,(E,theta) ,method='cubic')
        elif 'theta' in DF[j].keys():
            if j < 16:
                E=3.1
            else :
                E=2.9
            theta = DF[j]["theta"]
            Born1= griddata( (CST[j]['Eprime'],CST[j]['Theta']),CST[j]['Sig_Born'] ,(E,theta) ,method='cubic')
            CC= griddata( (CST[j]['Eprime'],CST[j]['Theta']),CST[j]['C_cor'] ,(E,theta) ,method='cubic')      
            RC= griddata( (CST[j]['Eprime'],CST[j]['Theta']),CST[j]['Sig_Rad'] ,(E,theta) ,method='cubic')      
          
            
     
        DF[j]['sigma'] =Born1#.diagonal()
        DF[j]['sigmaR'] =RC#.diagonal()
        DF[j]["C_cor"] =CC
        DF[j]['Sigma Data'] =DF[j]['sigma'] * DF[j]['D-mc ratio']
        DF[j]['Sigma Data err'] =DF[j]['sigma']* DF[j]['D-mc stat error']
        DF[j]['Sigma Data err noeff']=DF[j]['D-mc error noeff']*DF[j]['sigma']*DF[j]['D-mc ratio']
        
        DF[j]['Sigma Total err']=DF[j]['Sigma Data'] *(DF[j]['Total D-mc error']/DF[j]['D-mc ratio'])
        
        
        DF[j]=DF[j].dropna()
    
    return DF 

def EMC(FullA,atgt,a,FullB,btgt,b,comp):
    
    As=FullA[['Xbjc','Q2','D-mc ratio','D-mc stat error','Sigma Data','Sigma Data err'
              ,'Sigma Data err noeff','sigma','Thickness Err per']].copy()
    anames=As.keys() + atgt
    As.columns=anames
    As=As.rename(index=str,columns={"Xbjc{}".format(atgt):"Xbjc"})
    
    Bs=FullB[['Xbjc','Q2','D-mc ratio','D-mc stat error','Sigma Data','Sigma Data err'
              ,'Sigma Data err noeff','sigma','Thickness Err per']].copy()
    bnames=Bs.keys() + btgt
    Bs.columns=bnames
    Bs=Bs.rename(index=str,columns={"Xbjc{}".format(btgt):"Xbjc","Q2{}".format(btgt):"Q2"})
    
    #EMC=pd.merge(As,Bs, on="Xbjc")
    EMC= Bs.copy()
    
    for i in range(1,len(bnames)):
        EMC[anames.values[i]] = As[anames.values[i]].copy()
        
    EMC["EMC"] = (EMC["Sigma Data{}".format(atgt)]/a)/(EMC["Sigma Data{}".format(btgt)]/b) 
    EMC["EMC err"] = EMC["EMC"]*np.sqrt(pow(EMC['Sigma Data err noeff{}'.format(atgt)]/EMC['Sigma Data{}'.format(atgt)],2)+pow(EMC['Sigma Data err noeff{}'.format(btgt)]/EMC['Sigma Data{}'.format(btgt)],2)  )
   
    EMC["model EMC"] =(EMC["sigma{}".format(atgt)]/a)/(EMC["sigma{}".format(btgt)]/b) 
    fig = plt.figure(dpi=250,figsize=(11,5))
    ax=fig.add_subplot(1,1,1)
    
    
    if comp.find('3')  > -1:
        file="/home/jbane/emc_data/{}_maraEMC.dat".format(atgt,dtype=float)
        MARemc=pd.read_csv(file)
        MARemc.plot(ax=ax,x='x',y='ratio',yerr='error',kind='scatter',marker='o',color='navy',label="MARATHON yield ratio method")
    
    
    
    if comp.find('2')  > -1:
        E03103df={}
        E03color=['lightsalmon','salmon','crimson','coral','orangered']
        files = glob.glob('/home/jbane/emc_data/{}_q2*.dat'.format(atgt))
     
        for i in range(len(files)):
            file=files[i]
            Q2=file[file.find('q2')+3:file.find('q2')+6]
            header =['x','xi','Q2','W2','Ratio','Stat','Sys','Iso. corr.','Coul. corr.']
            exclude = [i for i, line in enumerate(open(file)) if not line.startswith(' 0') or line.startswith('** ') ]
            E03103df[i] = pd.read_table(file,delim_whitespace=True,header=None,names=header,skiprows = exclude[0:],dtype=float)
            
            E03103df[i]=E03103df[i].drop(index=0).copy()
            E03103df[i]['Ratio_n_iso'] =E03103df[i]['Ratio']/E03103df[i]['Iso. corr.']
            E03103df[i]['err'] = E03103df[i]['Ratio']*np.sqrt( pow(E03103df[i]['Stat'],2) + pow(E03103df[i]['Sys'],2))
            E03103df[i].plot(ax=ax,x='x',y='Ratio_n_iso',yerr='err',kind='scatter',marker='1',c=E03color[i],label="E03103 Q2={}".format(Q2))


            
    EMC['Thickness Error'] = np.sqrt(EMC["Thickness Err perD2"]**2 +EMC["Thickness Err per{}".format(atgt)]**2)
            
            
    EMC.plot(ax=ax,y='EMC',x="Xbjc",yerr="EMC err",kind='scatter',label="{}/{} cross section ratio".format(atgt,btgt))
    ax.grid("True")
    ax.set_xlabel("$X_{BJ}$")
    ax.set_ylabel("$\sigma({})$ / $\sigma({})$ ({}/{})  ".format(atgt,btgt,b,a))
    ax.set_xlim(0.15,0.85)
    ax.set_ylim(0.8,1.2)
    
    fig.text(0.5, 0.5, 'Preliminary',fontsize=75, color='gray',ha='center', va='center', alpha=0.5)
    fig.savefig('/home/jbane/latex_shows/images/{}_EMC.eps'.format(atgt),bbox_inches='tight')
    fig.savefig('/home/jbane/latex_shows/images/{}_EMC.pdf'.format(atgt),bbox_inches='tight')

    return EMC
 
def multiplot(DF,x,y,yerr="zero",labels=[""]):
    
    if yerr == "zero":
        yerr = 0
    fig =plt.figure(dpi=250,figsize=(7,5))
    ax=fig.add_subplot(1,1,1)
    
    if len(labels) != len(DF.keys()) :
        labels = [""]* len(DF.keys())
    #markers[i]
    j=0  
    for i in DF.keys():
        DF[i].plot(ax=ax,x=x,y=y,yerr=yerr,kind='scatter', marker=mar[i],color=colors[i],label=labels[j])
        j=j+1
        
    ax.grid("True")
def Bornplot(DF,tgts,mflag=1):
    fig =plt.figure(dpi=750,figsize=(7,5))
    ax=fig.add_subplot(1,1,1) 
    mar=["+","x","1","2"]
    
    
    for i in range(len(DF)):
        
        F= interpolate.interp1d(DF[i]['Xbjc'],DF[i]['sigma'], kind='cubic')
       #F= interpolate.Rbf(DF[i]['Xbjc'],DF[i]['sigma'])
       # y1=DF[i]['sigma']*0.999
       # y2=DF[i]['sigma']*1.001
        
        A=DF[i]['Xbjc'].keys()
        
        xf=DF[i]['Xbjc'][A[0]]*1.001
        xl=DF[i]['Xbjc'][A[size(DF[i]['Xbjc'])-1]]*0.99999
        ss = (xl-xf)/1000
        #print(A, xf,xl)
        x=np.arange(xf,xl,ss)
        #print(x)
        y1= F(x)*0.999
        y2= F(x)*1.001
        
        
        #print(i,colors[len(colors)-i-1], colors[i+len(DF)], colors[i])
        

        if mflag is 1:
            ax.fill_between(interpolate="True",x=x,y1=y1,y2=y2,color=colors[len(colors)-i-1],label="Model {}".format(tgts[i]))
        name= "{}".format(tgts[i]) 
        
        (DF[i].plot(ax=ax,x="Xbjc",y='Sigma Data',kind='scatter',marker=mar[i]
                   ,color=colors[i],yerr='Sigma Total err',label=name
                   ,linewidth=2, s=8))
        #DF[i].plot(ax=ax,x="Xbjc",y='Sigma Data',kind='scatter',marker=markers[i+len(DF)]
        #,color=colors[i+len(DF)+1],yerr='Sigma Data err',label="Data$\sigma$ {} 
        #-stat error".format(tgts[i]))
        
    ax.set_title("Cross Section")
    ax.set_yscale("log")
    ax.set_ylabel("$\sigma$")
    ax.set_xlabel("$X_{Bj}$")
    ax.grid(True)
    ax.legend(ncol=3)
        
    ax.legend()
    return fig
def ISO_Cor_EMC2(EMC_A,tgt_name,A_num,Z_num):
    
    #fig= plt.figure(dpi=500,figsize=(11,9))
    #plt.title('Cross section ratio Isoscalar corrected')
    #x=fig.add_subplot(1,1,1)
    He3df_cor_mined = pd.read_csv("/home/jbane/Downloads/corrected.csv",names=['xbj','EMC'])
    He3df_uncor_mined = pd.read_csv("/home/jbane/Downloads/uncorrected.csv",names=['xbj','EMC'])
    
    
    #He3_q2_5.3_ratio.dat
    #cols=["x","xi","Q2","W2","Ratio","Stat","Sys","Iso. corr.","Coul. corr."]
    
    
   # Fn_Fp= He3df_cor_mined
   # Fn_Fp['Fn/Fp']=  He3df_uncor_mined['EMC']/ He3df_cor_mined['EMC']
   # F_n_p = interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Fn/Fp'],kind='linear'
#, copy=True , bounds_error=False)
    
    files = '/home/jbane/emc_data/He3_q2_5.3_ratio.dat'
    file=files
    header =['x','xi','Q2','W2','Ratio','Stat','Sys','Iso. corr.','Coul. corr.']
    exclude = ([i for i, line in enumerate(open(file)) if not line.startswith(' 0') 
               or line.startswith('** ') ])
    E03103df =( pd.read_table(file,delim_whitespace=True,header=None,names=header
                             ,skiprows = exclude[0:],dtype=float))
    E03103df=E03103df.drop(index=0).copy()
    Isocor=(interpolate.interp1d(E03103df['x'],E03103df['Iso. corr.'],kind='linear'
                                ,copy=True,bounds_error=False))
    
  
    Fn_Fp= pd.read_csv("/home/jbane/Downloads/Fn_Fp.csv")
    F_n_p=( interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Fn/Fp'],kind='linear'
                                , copy=True, bounds_error=False ) )
    F_err= (interpolate.interp1d(Fn_Fp['xbj'],Fn_Fp['Total_Error'],kind='linear'
                                , copy=True , bounds_error=False))

    #p=[1.019,-1.317,0.8194,0.0]
    p=[0.910,-1.07,0.586,0.012]
    function_fn_fp= lambda x: x**3*p[3]+x**2*p[2] + x*p[1] + p[0]
    
    EMC_A['Iso_fac_MARA']= (0.5*(1.0 + function_fn_fp(EMC_A['Xbjc'])))/((1/A_num)*(Z_num+(A_num-Z_num)*function_fn_fp(EMC_A['Xbjc'])))
    EMC_A['Iso_EMC_MARA'] =EMC_A['EMC']*EMC_A['Iso_fac_MARA']
    
    EMC_A['Iso_fac'] = (0.5*(1.0 + F_n_p(EMC_A['Xbjc'])))/((1/A_num)*(Z_num+(A_num-Z_num)*F_n_p(EMC_A['Xbjc'])))
    EMC_A['Iso_EMC'] =EMC_A['EMC']*EMC_A['Iso_fac']
    
    EMC_A['Iso_mEMC_MARA'] =EMC_A['model EMC']*EMC_A['Iso_fac_MARA']
    EMC_A['Iso_mEMC'] =EMC_A['model EMC']*EMC_A['Iso_fac']  
    
    """John As F2 ratio"""
    kpx=EMC_A['Xbjc']
    kpq=EMC_A['Q2']
 
    JA_mar=Rnp(kpx,kpq)
    JA_mar=np.array(JA_mar)
    JA_err_fun =lambda x: 0.11*np.exp(-12*(1-x)) + 0.014
    JA_mar_err= JA_err_fun(kpx)
    JA_mar_rel_err= JA_mar_err/JA_mar
 
    EMC_A['Iso_fac_JA'] =(0.5*(1.0 + JA_mar))/((1/A_num)*(Z_num+(A_num-Z_num)*JA_mar))
    EMC_A['Iso_EMC_JA'] =EMC_A['EMC']*EMC_A['Iso_fac_JA']
    #EMC_A['Iso_EMC'] =EMC_A['EMC']*Isocor(EMC_A['Xbjc'])
 
    ##ADD in aditional error from correction
    #EMC_A.plot( ax=x,x="Xbj",xerr='offset',y="Iso_EMC",yerr='EMC_err',
    #kind="scatter",color='g',marker="x",label="{}/D Iso".format(tgt_name))
    #x.set_ylim(0.75,1.2)
    #x.grid(True)
    return EMC_A, He3df_cor_mined
        

def ErrorTable(DF):
    name=(["Xbjc","Stat Err",'Yield' ,"PC Err", 'PC', 'Eff err', 'Effs',
          'Dens Cor. Err per','Thickness Err per',"Beta Decay Error per.",
          'MC_Error',"MC_Yield","ECC Err",'Charge Err per',
          'Sigma Data err noeff','Sigma Total err','Sigma Data'])
    
    DFs=DF[name].copy()
    DFs['Stat Err per']=round(DF["Stat Err"]/DF["Yield"]*100,3)
    DFs['PC Err per']=round(DF["PC Err"]/DF["PC"]*100,3)
    DFs['Eff Err per']=round(DF["Eff err"]/DF["Effs"]*100,3)
    DFs['Data err noeff']=(round(DF["Sigma Data err noeff"]
                                /DF["Sigma Data"]*100,3))
    DFs["MC Err per"] = round(DFs['MC_Error']/DFs["MC_Yield"] *100,3)
    
    DFs['Data err']=(round(DF["Sigma Total err"]
                                /DF["Sigma Data"]*100,3))
    DFs["Beta Decay Error per."] =round(DF["Beta Decay Error per."]*100,3)
    DFs['Dens Cor. Err per'] =round(DF['Dens Cor. Err per']*100,3)
    DFs['ECC Err per'] =round(DF['ECC Err']*100,3)
    
    
    
    namefs=(["Stat Err",'Yield',"Stat Err per" ,"PC Err", 'PC', 'PC Err per'
            ,'Eff err', 'Effs', 'Eff Err per'])
    names=(["Xbjc","Stat Err per" ,'PC Err per','ECC Err per', 'Eff Err per','Charge Err per',
           'Dens Cor. Err per','Thickness Err per',"Beta Decay Error per.",'MC Err per',
           'Data err noeff','Data err'])
    DFs=round(DFs[names],3)
    DFs=(DFs.rename(index=str, columns={"Xbjc":"X"
                                       ,"Stat Err per":"Satistical Error" 
                                       ,'PC Err per':'Positron Correction Error'
                                       ,'ECC Err per':'Endcap Correction Error'
                                       ,'Eff Err per':'Efficiency Error'
                                       ,'Charge Err per':'Charge Error'
                                       ,'Dens Cor. Err per':'Density Correction Error'
                                       ,'Thickness Err per':'Target Tickness Error'
                                       ,"Beta Decay Error per.":"Beta Decay"
                                       ,"MC Err per":'MonteCarlo Error'
                                       ,'Data err noeff':'Relative EMC Error'
                                       ,'Data err':'Relative Cross Section Error'}))
    
    
    return DFs
    
    
    
    
def Rnpe(epls):
    p=[0.0,0.816,-0.661,0.184,5.509,-0.034,8.714,-0.072,0.450]
    return( [(p[1] +  p[2]*epl) + p[3]*np.exp(-p[4]*epl) +  p[5]*np.exp(-p[6]*(1-epl)) + 
            p[7]* pow(max(0,epl-p[8]),2) for epl in epls])

def Rnp(x,Q2):
    epls = eplfun(x,Q2)
    p=[0.0,0.816,-0.661,0.184,5.509,-0.034,8.714,-0.072,0.450]
    return ([(p[1] +  p[2]*epl) + p[3]*np.exp(-p[4]*epl) + p[5]*np.exp(-p[6]*(1-epl)) + 
            p[7]* pow(max(0,epl-p[8]),2) for epl in epls])

def eplfun(A,Q2):
    '''epsilon function: takes x and Q2 returns epsilon'''
    if type(Q2) != type(A):
        Q2=np.full(len(A),Q2)
    B=np.column_stack((A,Q2))
    
    return [( 2*x) /(1 +np.sqrt(1 + (4*pow(0.989,2)*pow(x,2)/Q))) for x,Q in B ]


def ISOcor_factor(np_R,a,z,x,newx):
    # This taks a Fn/Fp at x ratio and a and z to to calculate iso cor factor
    A=(0.5*(1.0 + np_R) /((1/a)*(z+(a-z)*np_R)))
    
    kind='cubic'
    if len(x) ==2:
        kind='linear'
    if len(x) >1:      
        B=interpolate.interp1d(x,A,kind=kind,copy=True, bounds_error=False) 
        return B(newx)
    else:
        return A
                  

def ISOcor(EMC,fact):
    
    return EMC*fact

def NMC_F2np(x,Q2):
    """F2 ratio from NMC paper 1992"""
    AX=0.979-1.692*x+2.797*x**2-4.313*x**3+3.075*x**4
    BX=-0.171*x+0.244*x**2
    F2NP_NMC=AX*((Q2/20.0)**BX)*(1+x**2/Q2)
    
    return F2NP_NMC


    

In [3]:
def He3ContKin(kin,suf="all"): 
    """
        fH = sum(QifHi)
    fHi= hep/(tp+hep)
    hep = nh(0) + nt(0)*(1-exp(-t/tau))
    tau = L(.5)/ln(2)
    """
    daytosec = 60*60*24
    halflife = 4500 *daytosec
    deltahl  = 8    *daytosec
    
    meanlife = halflife/np.log(2.0)
    deltaml  = deltahl/np.log(2.0)
    
    TritiumMag_0     = 1.54E22
    DeltaTritiumMag_0= 0.02E22
    
    HeliumMag_0     = 4.51E18
    DeltaHeliumMag_0= 0.90E18
    deltaW=0.0
    deltaQ = 0.005
    Tot_0 = HeliumMag_0 + TritiumMag_0

    runlist = SQLF.SQLRuns("H3",kin,suf)

    totalcharge=0
    totalWeight=0
    
    for i in runlist:
        run=runlist[i]
        charge_run = SQLF.SQLCharge(run,0)
        totalcharge = totalcharge+charge_run[0]

        runtime = timefromfill(run)
        
        expf=np.exp(-runtime/meanlife)

        HeliumMag_run  =HeliumMag_0 + TritiumMag_0*(1 - np.exp(-runtime/meanlife))  
        DeltaHeliumMag_run = sqrt(pow(DeltaHeliumMag_0,2) + 
                                  pow((1 - np.exp(-runtime/meanlife) )*DeltaTritiumMag_0,2) )

        TritiumMag_run =TritiumMag_0*(np.exp(-runtime/meanlife))
        DeltaTritiumMag_run = (np.exp(-runtime/meanlife)) *DeltaTritiumMag_0

        HeliumFrac_run = HeliumMag_run / (TritiumMag_0 + HeliumMag_0 )
        DeltaHeFrac_run = sqrt( pow( ((1-expf)/Tot_0  - HeliumMag_run/pow(Tot_0,2)) 
                                    * DeltaTritiumMag_0 ,2) 
                               + pow( ( 1/Tot_0 - HeliumMag_run/pow(Tot_0,2)) 
                                     * DeltaHeliumMag_0,2) )
        
       
        
        RunW = charge_run[0] * HeliumFrac_run
        totalWeight = RunW + totalWeight
                             
        deltaW = pow(deltaQ,2) + DeltaHeFrac_run**2
        
            
    deltaW= sqrt(deltaW)
        
    TritiumFact = totalcharge/ ( totalcharge + totalWeight)
    HeliumFact  = totalWeight/ ( totalcharge + totalWeight)
    
    Fullcor = totalWeight/totalcharge
    
    dFd = np.sqrt(totalcharge*0.005 + deltaW**2)
    Fd =( totalcharge + totalWeight)
        
    deltaT =TritiumFact*sqrt( (deltaQ)**2 + (dFd/Fd)**2)
    deltaH =HeliumFact*sqrt( (deltaW/totalWeight)**2 + (dFd/Fd)**2)
    
    return TritiumFact,HeliumFact,deltaT,deltaH, Fullcor

    
def He3Cont(H3,He3,suf="all"):
    """
    YT = YH3( Qtot / (Qtot - fH)) - YH( fH / (Qtot - fH)
    """
    for kin in H3:
        Betaf = He3ContKin(kin,suf)
        H3[kin]['Cont. Yield'] = H3[kin]['Yield'].copy()
        H3[kin]['Yield'] = H3[kin]['Cont. Yield'] * Betaf[0] - He3[kin]['Yield']*Betaf[1]
    
        deltaA  = H3[kin]['Cont. Yield'] * Betaf[0]*np.sqrt((Betaf[2]/Betaf[0])**2 + ( H3[kin]['Total D-mc error']/H3[kin]['D-mc ratio'])**2)
        deltaB  = He3[kin]['Yield']*Betaf[1]*np.sqrt((Betaf[3]/Betaf[1])**2 + (He3[kin]['Total D-mc error']/He3[kin]['D-mc ratio'])**2)
    
        H3[kin]
        H3[kin]['Total D-mc error'] = np.sqrt(deltaA**2 +deltaB**2)
    
    return H3
       

In [4]:
def KP():
    files='/home/jbane/emc_data/F2dis_os1tm1ht1mec1_Dav18_He3Salme'
    file=files
    p=[1.019,-1.317,0.8194]

    KP = pd.read_table(file,delim_whitespace=True,dtype=float)
    KP['EMC_He3'] = KP['F2He3']/KP['F2D']
    KP['EMC_H3'] = KP['F2H3']/KP['F2D']
    KP['Fn/Fp']=KP['F2neutron']/KP['F2proton']

    F_n_p= interpolate.interp1d(KP['x'],KP['Fn/Fp'],kind='linear', copy=True, bounds_error=False ) 
    f_fn_fp= lambda x: x**2*p[2] + x*p[1] + p[0]

    KP['He3_Iso_fac'] = (0.5*(1.0 + f_fn_fp(KP['x']))) /((1/3)*(2+(3-2)*f_fn_fp(KP['x'])))
    KP['He3_Iso_EMC'] =KP['EMC_He3']*KP['He3_Iso_fac']
    
    KP['He3_Iso_fac_KP'] = (0.5*(1.0 + F_n_p(KP['x']))) /((1/3)*(2+(3-2)*F_n_p(KP['x'])))
    KP['He3_Iso_EMC_KP'] =KP['EMC_He3']*KP['He3_Iso_fac_KP']
    

    KP['H3_Iso_fac'] = (0.5*(1.0 + f_fn_fp(KP['x']))) /((1/3)*(1+(3-1)*f_fn_fp(KP['x'])))
    KP['H3_Iso_EMC'] =KP['EMC_H3']*KP['H3_Iso_fac']
    
    
    KP['H3_Iso_fac_KP'] = (0.5*(1.0 + F_n_p(KP['x']))) /((1/3)*(1+(3-1)*F_n_p(KP['x'])))
    KP['H3_Iso_EMC_KP'] =KP['EMC_H3']*KP['H3_Iso_fac_KP']
    
    
    return KP

In [5]:
def E03He3():
    files = '/home/jbane/emc_data/He3_q2_5.3_ratio.dat'
    file=files

    header =['x','xi','Q2','W2','Ratio','Stat','Sys','Iso. corr.','Coul. corr.']
    exclude = [i for i, line in enumerate(open(file)) if not line.startswith(' 0') or line.startswith('** ') ]
    E03103df = pd.read_table(file,delim_whitespace=True,header=None,names=header,skiprows = exclude[0:],dtype=float)

    E03103df=E03103df.drop(index=0).copy()

    E03103df['EMC']=E03103df['Ratio']
    E03103df['EMC_noiso']=E03103df['EMC']/E03103df['Iso. corr.']
    E03103df['Total error']=sqrt ( pow(E03103df['Stat']/E03103df['EMC'],2) 
                                  + pow(E03103df['Sys']/E03103df['EMC'],2) )*E03103df['EMC']
    
    return E03103df




       

In [4]:
 
def modelEMC(tgtA,A,Za,xbj,Qsqr):
    EMC=pd.DataFrame()
    EMCxbj=[]

    n=xbj.keys()
    l=len(n)
    f=xbj[n[0]]
    last=xbj[n[l-1]]
    x=np.arange(f,last,0.001)
   
    
    if len(xbj) == 2 and len(Qsqr) == 2:
        Qf = interpolate.interp1d(xbj,Qsqr,
                              copy=True, bounds_error=False,fill_value="extrapolate" ) 
        
        Q2=Qf(x)
    elif len(xbj) > 2 and len(Qsqr) > 2:
        Qf = interpolate.interp1d(xbj,Qsqr,kind='cubic',
                              copy=True, bounds_error=False,fill_value="extrapolate" ) 
        
        Q2=Qf(x)    
    else:
        Q2=Qsqr
        


    file_name="/home/jbane/tritium/T2_externals/OUT/marathon_kin100_{}.out".format(tgtA)
    Adf=pd.read_table(file_name,delim_whitespace=True)   

    tgtD2="D2"
    file_nameD2="/home/jbane/tritium/T2_externals/OUT/marathon_kin100_{}.out".format(tgtD2)
    D2df=pd.read_table(file_nameD2,delim_whitespace=True)   
    
    
    
    if len(xbj) > 1 and len(Qsqr) > 1:
    
        BornA= griddata( (Adf['x'],Adf['Q2']),Adf['Sig_Born'] ,(x,Q2) 
                    ,method='cubic')
        BornD2= griddata( (D2df['x'],D2df['Q2']),D2df['Sig_Born'] ,(x,Q2) 
                    ,method='cubic')
    
    else:
        BornA=1
        BornD2=1
    
    EMC['Xbjc'] = x
    EMC["EMC"]=BornA/BornD2 * (2/A)
    #    EMCxbj=pd.Series(x)

    EMC['Q2'] = Q2

    return EMC.dropna()
    
def EMCfakeD(Adf,Atgt, A):
    
    xbj=Adf["Xbjc"]
    Q2=Adf["Q2"]

    #Get some D cross section
    tgtD2="D2"
    file_nameD2="/home/jbane/tritium/T2_externals/OUT/marathon_kin100_{}.out".format(tgtD2)
    D2df=pd.read_table(file_nameD2,delim_whitespace=True) 
    Adf["fD"]= griddata( (D2df['x'],D2df['Q2']),D2df['Sig_Born'] ,(xbj,Q2) ,method='cubic')
    
    Adf["EMC_aD"] = Adf["Sigma Data{}".format(Atgt)]/Adf["fD"] * (2/A)
    Adf["EMC_aD_Iso_MARA"]=Adf['Iso_fac_MARA'] * Adf["EMC_aD"]
    
    aerr=Adf['Sigma Data err noeff{}'.format(Atgt)]/Adf['Sigma Data{}'.format(Atgt)] 
    derr=0.0
    Adf["EMC err aD"]= sqrt(aerr**2 + derr**2) * Adf["EMC_aD"]
    
    a=Adf['Thickness Err per{}'.format(Atgt)]
    b=0.0
    Adf["Thickness Err No D"] = sqrt(a**2+b**2)
    
    
    return Adf 
     
    
    

In [7]:
def deltatheta(i):

    tph=-0.02
    dtph=0.002
    tth=-0.00
    dtth=0.0006
    theta= whatangle(i)

    A=cos(torad(theta)) - tph*sin(torad(theta))
    dA = sqrt( pow(sin(torad(theta)),2) * pow(dtph,2) )

    B=sqrt(1+pow(tth,2) + pow(tph,2))
    dB = sqrt(pow(tth/B,2) * pow(dtth,2) + pow(tph/B,2) * pow(dtph,2))

    C=A/B
    dC =C*sqrt( pow(dA/A,2) +pow(dB/B,2) )

    ntheta = todeg( arccos(A/B))
    dntheta= todeg( dC/sqrt(1 +C*C)) 


    return dntheta

def propci_wilson_cc(count, nobs, alpha=0.05):
    # get confidence limits for proportion
    # using wilson score method w/ cont correction
    # i.e. Method 4 in Newcombe [1]; 
    # verified via Table 1
    from scipy import stats
    n = nobs
    p = count/n
    q = 1.-p
    z = stats.norm.isf(alpha / 2.)
    z2 = z**2   
    denom = 2*(n+z2)
    num = 2.*n*p+z2-1.-z*np.sqrt(z2-2-1./n+4*p*(n*q+1))    
    ci_l = num/denom
    num = 2.*n*p+z2+1.+z*np.sqrt(z2+2-1./n+4*p*(n*q-1))
    ci_u = num/denom
    if p == 0:
        ci_l = 0.
    elif p == 1:
        ci_u = 1.
    return ci_l-p, ci_u-p

In [1]:
def NewBorn(DF,tgt,v="Xbjc"):

    #Read in a Cross section table into CST, this will be an array one element for each kin
    CST={}
    Cross_table(tgt,CST)

    for i in DF:
           
        
        ep=3.1 #Left arm centeral momentum 
        m=0.93828 #mass of proton
        th=whatangle(i)        # Take the central angle of the kinematic
        thr=torad(th)          # turn that into radians
        #short had for sin(th/2)**2
        sinth=sin(thr/2.0)**2
        # Using resolution numbers from HAll A nim and other calculations
        de =0.0005 #nim A pg  
        dep=0.0001 
        
        phi    =0.05
        theta  =0.0
        theta_0=thr

        dphi    =0.0002
        dtheta  =0.0006
        dtheta_0=0.00008


        beta = phi*cos(theta_0) + cos(theta_0)
        lamb = 1+phi**2+theta**2
        alpha= np.sqrt(1 - (beta**2/lamb))

        diff1= - (phi*cos(theta_0) - sin(theta_0))/(np.sqrt(lamb)*alpha)* dtheta_0
        diff2= ( sin(theta_0)/np.sqrt(lamb) - phi*beta/lamb**(3/2) ) * dphi
        diff3= (phi*beta / (lamb**(3/2)*alpha)) *dtheta

        dscat_theta = np.sqrt(diff1**2 +diff2**2 +diff3**2)
        dth=dscat_theta
        
        
        dthr= 0.0006#dth

        ne = e+de
        nep= ep+dep
        nthr=thr+dthr

        # grab QQ and x from the Dataframe for this kinematic
        QQ = DF[i]["Q2"]
        x  = DF[i]["Xbjc"]
        # use those QQ and x to calc CS
        Born= griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['Sig_Born'] ,(x,QQ) ,method='cubic')
        CC  = griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['C_cor']    ,(x,QQ) ,method='cubic')
        RC  = griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['Sig_Rad']  ,(x,QQ) ,method='cubic')

        #Calulate change and new QQ and x for de
        QQ_de = ((QQ/e) *de)
        QQ_ne = QQ_de+QQ
        x_de  = (-(x/e) * (ep/(e-ep))*de)
        x_ne  = x + x_de

        #Calulate change and new QQ and x for dep
        QQ_dep = ((QQ/ep) *dep)
        QQ_nep = QQ_dep+QQ
        x_dep  = ((x/ep) * (e/(e-ep))*dep)
        x_nep  = x + x_dep

        #Calulate change and new QQ and x for dth
        QQ_dth = ((QQ/tan(thr/2) ) *dthr)
        QQ_nth = QQ_dth+QQ
        x_dth  = ((x/tan(thr/2)) * dthr)
        x_nth  = x + x_dth

      #use look up table to get born columb c. and radiative correction for de,dep,dth
        Born_de= griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['Sig_Born'] ,(x_ne,QQ_ne) ,method='cubic')
        CC_de  = griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['C_cor']    ,(x_ne,QQ_ne) ,method='cubic')
        RC_de  = griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['Sig_Rad']  ,(x_ne,QQ_ne) ,method='cubic')

        Born_dep= griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['Sig_Born'] ,(x_nep,QQ_nep) ,method='cubic')
        CC_dep  = griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['C_cor']    ,(x_nep,QQ_nep) ,method='cubic')
        RC_dep  = griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['Sig_Rad']  ,(x_nep,QQ_nep) ,method='cubic')

        Born_dth= griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['Sig_Born'] ,(x_nth,QQ_nth) ,method='cubic')
        CC_dth  = griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['C_cor']    ,(x_nth,QQ_nth) ,method='cubic')
        RC_dth  = griddata( (CST[i]['x'],CST[i]['Q2']),CST[i]['Sig_Rad']  ,(x_nth,QQ_nth) ,method='cubic')

        #Calc. err for born on each var,
        de_err  = abs(Born_de-Born)/Born   
        dep_err = abs(Born_dep-Born)/Born   
        dth_err = abs(Born_dth-Born)/Born   

        total_err = sqrt( de_err**2+ dep_err**2 + dth_err**2)
        DF[i]['Born_de'] = Born_de
        DF[i]['Born_dep'] = Born_dep
        DF[i]['Born_dth'] = Born_dth
        DF[i]['Res Error Per'] = total_err
    return DF

In [8]:
def NewXQ(DF,v="Xbjc"):
    
       
    for i in DF:
        nxar=[]
        nqar=[]
        kk=0
        indy=DF[i].index
        for j in DF[i][v]:
            k=indy[kk]
            e=10.6
            if v == "E`":
                ep = j
            else:
                ep=3.1
            if v == 'theta':
                th=j
            else:
                th=whatangle(1)
                
            thr=torad(th)
            

            de =0.0005
            dep=0.00025
            dth= 0.0015*th
            dthr= 0.0015*thr

            nthr=thr+dthr
            ne = e+de
            nep= ep+dep

            qq1=4*(e+de)*(ep+dep)*sin( (thr+dthr)/2.0 )**2
            x1=qq1/ (2 * 0.93828* ((e+de)-(ep+dep)))      

            if v =="Xbjc":
                qq=DF[i]['Q2'][k]
                x=j
            else:
                qq=4*(e)*(ep)*sin( thr/2.0 )**2
                x=qq/ (2 * 0.93828* ((e) -(ep))) 

            dqq = qq*sqrt( (de/e)**2 + (dep/ep)**2  + (cos(thr/2)*sin((thr)/2)*dthr)**2 )
            dx = x*sqrt( (dqq/qq)**2 + ( (sqrt(de + dep) )/(e-ep))**2 )   
            
            if v=="Xbjc":
                nqar.append(qq+dqq)
                nxar.append(x+dx)
            else:
                nqar.append(qq1)
                nxar.append(x1)
            kk=kk+1
        DF[i]['nX'] = nxar
        DF[i]['nQ2'] = nqar
        
        
    return DF


def ONewBorn(DF, tgt,x,y):
    
    CST={}
    DFBorn1={}
    DFBorn2={}
    DFBorn3={}
    Cross_table(tgt,CST)
    
    KT=kintable()
    for j in DF.keys():
        
        if type(x) == str:
            xaxis=DF[j][x]
            yaxis=DF[j][y]
        else:
            xaxis=x
            yaxis=y
  
        Born1= griddata( (CST[j]['x'],CST[j]['Q2']),CST[j]['Sig_Born'] ,(xaxis,yaxis) ,method='cubic')
        CC = griddata( (CST[j]['x'],CST[j]['Q2']),CST[j]['C_cor'] ,(xaxis,yaxis) ,method='cubic')
        RC = griddata( (CST[j]['x'],CST[j]['Q2']),CST[j]['Sig_Rad'] ,(xaxis,yaxis) ,method='cubic')
     
        DF[j]['nsigma'] =Born1#.diagonal()
        DF[j]["nC_cor"] =CC
        DF[j]['nSigma Data'] =DF[j]['sigma'] * DF[j]['D-mc ratio']
        DF[j]["nsigmaR"] = RC
        
                
        DF[j]["Res Error Per O"] = abs(DF[j]['nsigma'] -DF[j]['sigma'])/DF[j]['sigma']
                
        DF[j]=DF[j].dropna()
   
    return DF 



In [1]:
def modelD(DF,tgt):
    #tgt="D2"
    #DF=D2Full_4

    basefname="/home/jbane/tritium/Hanjie_T2_externals/Radiative-correction/T2_externals_clean/OUT/"
    CStable111 = pd.read_table("{}{}_kin100_111_xs.out".format(basefname,tgt), delim_whitespace=True)

    CStable111=CStable111.rename(index=str,columns={"***x":"x"})

    xbj=DF["Xbjc"]
    Q2=DF["Q2"]
    Born111= griddata( (CStable111['x'],CStable111['Q2']),CStable111['Sig_Born'] ,(xbj,Q2) ,method='cubic')
    k=0
    
    for i in Born111:
        if i != i:
            if k not in xbj.index: break
            x=xbj[k]
            qq=Q2[k]
            ep = 10.6 -1/(x*2*0.938/qq) 
            sin=sqrt(qq/(4*ep*10.6))
            thr=2*arcsin(sin)
            th=todeg(thr)
            Born111[k] = griddata( (CStable111['Eprime'],CStable111['Theta']),CStable111['Sig_Born'] 
                                  ,(ep,th) ,method='cubic')
            if Born111[k] != Born111[k]:
                Born111[k] = griddata( (CStable111['x'],CStable111['Q2']),CStable111['Sig_Born'] 
                                  ,(xbj[k],Q2[k]) ,method='nearest')
        k=k+1

    DF['Born111']=Born111


    CStable123 = pd.read_table("{}{}_kin100_123_xs.out".format(basefname,tgt), delim_whitespace=True)
    CStable123=CStable123.rename(index=str,columns={"***x":"x"})
    Born123= griddata( (CStable123['x'],CStable123['Q2']),CStable123['Sig_Born'] ,(xbj,Q2) ,method='cubic')
    k=0
    for i in Born123:
        if i != i:
            if k not in xbj.index: break
            x=xbj[k]
            qq=Q2[k]
            ep = 10.6 -1/(x*2*0.938/qq) 
            sin=sqrt(qq/(4*ep*10.6))
            thr=2*arcsin(sin)
            th=todeg(thr)
            Born123[k] = griddata( (CStable123['Eprime'],CStable123['Theta']),CStable123['Sig_Born'] 
                                  ,(ep,th) ,method='cubic')
            if Born123[k] != Born123[k]:
                Born123[k] = griddata( (CStable123['x'],CStable123['Q2']),CStable123['Sig_Born'] 
                                  ,(xbj[k],Q2[k]) ,method='nearest')
        k=k+1

    DF['Born123']=Born123

    CStable211 = pd.read_table("{}{}_kin100_211_xs.out".format(basefname,tgt), delim_whitespace=True)
    CStable211=CStable211.rename(index=str,columns={"***x":"x"})
    Born211= griddata( (CStable211['x'],CStable211['Q2']),CStable211['Sig_Born'] ,(xbj,Q2) ,method='cubic')
    k=0
    for i in Born211:
        if i != i:
            if k not in xbj.index: break
            x=xbj[k]
            qq=Q2[k]
            ep = 10.6 -1/(x*2*0.938/qq) 
            sin=sqrt(qq/(4*ep*10.6))
            thr=2*arcsin(sin)
            th=todeg(thr)
            Born211[k] = griddata( (CStable211['Eprime'],CStable211['Theta']),CStable211['Sig_Born'] 
                                  ,(ep,th) ,method='cubic')
            if Born211[k] != Born211[k]:
                Born211[k] = griddata( (CStable211['x'],CStable211['Q2']),CStable211['Sig_Born'] 
                                  ,(xbj[k],Q2[k]) ,method='nearest')
        k=k+1

    DF['Born211']=Born211
    
    CStable122 = pd.read_table("{}{}_kin100_122_xs.out".format(basefname,tgt), delim_whitespace=True)
    CStable122=CStable122.rename(index=str,columns={"***x":"x"})
    Born122= griddata( (CStable122['x'],CStable122['Q2']),CStable122['Sig_Born'] ,(xbj,Q2) ,method='cubic')
    k=0
    for i in Born122:
        if i != i:
            if k not in xbj.index: break
            x=xbj[k]
            qq=Q2[k]
            ep = 10.6 -1/(x*2*0.938/qq) 
            sin=sqrt(qq/(4*ep*10.6))
            thr=2*arcsin(sin)
            th=todeg(thr)
            Born122[k] = griddata( (CStable122['Eprime'],CStable122['Theta']),CStable122['Sig_Born'] 
                                  ,(ep,th) ,method='cubic')
            if Born122[k] != Born122[k]:
                Born122[k] = griddata( (CStable122['x'],CStable122['Q2']),CStable122['Sig_Born'] 
                                  ,(xbj[k],Q2[k]) ,method='nearest')
        k=k+1

    DF['Born122']=Born122
    
    
    
    DF["Model Err Per"] = DF[["Born111","Born123","Born211"]] .std(axis=1,ddof=1)/DF["sigma"]


    return DF


In [ ]:
def inQuadDF(DF,A):
    C=0
    for a in A:
        C=C+pow(DF[a],2)
    C=sqrt(C)
    return C



def ErrorTables(DF):

    DFkeys =DF.keys()
    Allerrs =[]
    Yerrs=[]
    Oerrs=[]
    k=0
    BGerrslu=['PC',"ECC"]
    Yerrslu=['Stat',"Eff",'PC',"ECC","Dens Cor"]
    errs = ["err","Err","Error","error"]
    oerrslu=["MC_Error","Res","Model Err"]

    for i in DFkeys:

        for j in errs : 
            if j in i :
                Allerrs.append(i)
                break
    errkeys = DF[Allerrs].keys()
    for i in errkeys:
        for j in Yerrslu : 
            if j in i :
                Yerrs.append(i)
                break
        for j in oerrslu : 
            if j in i :
                Oerrs.append(i)
                break


    DFerrs=DF[["Xbjc",'Sigma Data']]

    DF_Yerrs= DF[["Xbjc",'Yield']]
    DF_Yerrs =pd.concat([DF_Yerrs,DF[Yerrs]],axis=1)
    DF_Yerrs["Stat Err"] =DF_Yerrs["Stat Err"]/DF_Yerrs["Yield"]
    DF_Yerrs["Corrected Norm. Yield Error"] = round(inQuadDF(DF_Yerrs,Yerrs),4)

    tmp=Yerrs.remove("PC Err")
    DF_Yerrs=DF_Yerrs.round({"PC Err":4})
    DF_Yerrs=DF_Yerrs.round({"Eff err":4})
    DF_Yerrs=DF_Yerrs.round({"Dens Cor. Err per":4})
    DF_Yerrs=DF_Yerrs.round({"Stat Err":4})
    DF_Yerrs.to_csv("/home/jbane/emc_data/YieldErrs.csv")


    DF_Oerrs=pd.concat([DFerrs,DF[Oerrs],DF['MC_Yield']],axis=1)
    DF_Oerrs["MC_Error"] =DF_Oerrs["MC_Error"]/DF["MC_Yield"]
    DF_Oerrs["Monte Carlo and Model Total Error"] = inQuadDF(DF_Oerrs,Oerrs)
    DF_Oerrs=DF_Oerrs.round(3)
    DF_Oerrs.to_csv("/home/jbane/emc_data/OtherErrs.csv")


    DF_sum=pd.concat([DFerrs,DF_Yerrs["Corrected Norm. Yield Error"],
                      DF_Oerrs["Monte Carlo and Model Total Error"]],axis=1)
    DF_sum["Cross Section Error"] = inquad(DF_Yerrs["Corrected Norm. Yield Error"],DF_Oerrs["Monte Carlo and Model Total Error"])
    DF_sum=DF_sum.round(3)
    DF_sum.to_csv("/home/jbane/emc_data/TotalErrs.csv")
    return DF_sum, DF_Yerrs, DF_Oerrs
    